In [1]:
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
import torch.nn.functional as F
import torch.nn as nn
import torch
import os
import sys
sys.path.append('/'+'/'.join(os.getcwd().split('/')[:-1])+'/')

from model.wrapped import FlattenM, MaxPool2d, Dropout, AdaptiveAvgPool2d
from model.blocks import ConvBnA, LinBnA, BasicBlock
from model.activations import ReLU, PACT
from model.Conversion import Start, Stop
from tqdm import tqdm
from model.QuantizationMethods.MinMSE import MinMSE_convolution_weight_quantization, MinMSE_linear_weight_quantization
from model.Quantizer import LinQuantExpScale
from model.sequential import Sequential
from model.linear import Linear


path = 'cifa10_resnet'


if not os.path.exists('./runs'):
    os.mkdir('./runs')
if not os.path.exists(f'./runs/{path}'):
    os.mkdir(f'./runs/{path}')

In [2]:
batch_size = 80
epochs = 50

In [3]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (.3, .3, .3))])
transform_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (.3, .3, .3))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
weight_quant = MinMSE_convolution_weight_quantization

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.start = Start(bits=8,size=(1,3,1,1),mode="auto",auto_runs=2)
        self.stop = Stop(size=(1,10))
        
        self.seq = Sequential(
            ConvBnA(  3, 128,3,1,1,activation=PACT,weight_quant=weight_quant),
            BasicBlock(128,128,1),
            Dropout(0.1),
            
            BasicBlock(128,256,2),
            Dropout(0.1),

            BasicBlock(256,512,2),
            Dropout(0.1),

            AdaptiveAvgPool2d((1,1)),
            FlattenM(1),
            LinBnA(512,10,weight_quant=MinMSE_linear_weight_quantization,weight_quant_channel_wise=True,activation=LinQuantExpScale,affine=False),
        )

    def forward(self, x):
        x = self.start(x)
        x = self.seq(x)
        x = self.stop(x)
        return x

In [5]:
def eval(net, pr=True):
    global testloader,device
    global best
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    net.eval()
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.to(device,non_blocking=True)
            labels = labels.to(device,non_blocking=True)
            # calculate outputs by running images through the network
            outputs = net(images)
            # loss = criterion(outputs, target)

            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    net.train()
    if pr:
        print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:4.1f} %')
    if best < 100 * correct / total:
        best = 100 * correct / total

    return 100 * correct / total, best


In [6]:
net = Net()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = net.to(device)


criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9,weight_decay=5e-4)
sched= optim.lr_scheduler.CosineAnnealingLR(optimizer,epochs,1e-5,verbose=False)

best = 0

In [7]:
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    training_running_correct = 0
    with tqdm(enumerate(trainloader, 0),total=len(trainloader),disable=False) as t:
        for i, (data,target) in t:
            # get the inputs; data is a list of [inputs, labels]
            target = target.to(device,non_blocking=True)
            data = data.to(device,non_blocking=True)

            # zero the parameter gradients
            optimizer.zero_grad(True)

            # forward + backward + optimize
            outputs = net(data)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()


            # print statistics
            running_loss += loss.item()
            # if i == 0:    # print every 2000 mini-batches
            _, preds = torch.sort(outputs.data.clone().detach(), 1,descending=True)
            training_running_correct += (preds[:,0].view(-1,1) == target.view(-1,1)).any(dim=1).sum().detach().item()
            t.set_postfix({'loss': running_loss/(i+1),'acc':training_running_correct/((i+1)*batch_size)})
    ev = eval(net,pr=False)
    print(f'[{epoch + 1:3d}, {i + 1:5d}] loss: {running_loss*batch_size/len(trainloader):6.3f},Train Acc: {training_running_correct/len(trainloader):3.1f}, Test Acc:{ev[0]:3.1f}%, Best test Acc:{ev[1]:3.1f}%')
    running_loss = 0.0
    torch.save(net.state_dict(),f"./runs/{path}/ckp.pt")
    sched.step()
            
print('Finished Training')


100%|██████████| 625/625 [00:56<00:00, 11.16it/s, loss=1.63, acc=0.501]


reduce autorun by 1: 1 min/max -1.6666666269302368 / 1.6666666269302368
[  1,   625] loss: 130.448,Train Acc: 40.1, Test Acc:60.6%, Best test Acc:60.6%


100%|██████████| 625/625 [00:53<00:00, 11.65it/s, loss=1.4, acc=0.632] 


reduce autorun by 1: 0 min/max -1.6666666269302368 / 1.6666666269302368
[  2,   625] loss: 112.004,Train Acc: 50.6, Test Acc:60.5%, Best test Acc:60.6%


100%|██████████| 625/625 [00:51<00:00, 12.06it/s, loss=1.3, acc=0.69]  


...

[ 48,   625] loss: 55.117,Train Acc: 79.8, Test Acc:90.1%, Best test Acc:90.4%


100%|██████████| 625/625 [00:52<00:00, 11.91it/s, loss=0.689, acc=0.998]


[ 49,   625] loss: 55.103,Train Acc: 79.8, Test Acc:90.2%, Best test Acc:90.4%


100%|██████████| 625/625 [00:53<00:00, 11.74it/s, loss=0.688, acc=0.997]


[ 50,   625] loss: 55.066,Train Acc: 79.8, Test Acc:90.1%, Best test Acc:90.4%
Finished Training
